In [68]:
# Import packages
import numpy as np
import pandas as pd
ps = np.finfo(float).eps
from numpy import log2 as log

In [69]:
# Load training data, split into train and validation sets
data = pd.read_csv("train.csv")
data = train_data[['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years', 'sales', 'salary', 'left']]
train_data = data.sample(frac=0.8,random_state=200)
val_data = data.drop(train_data.index)

In [70]:
# Find unique values for a column/attribute
def unique_vals(dataset, attribute):
    return dataset[attribute].unique()

In [71]:
def entropy_before_split(dataset):
    """ Finds Entropy of dataset before any split """
    
    dependent_variable = dataset.keys()[-1]
    entropy = 0
    labels = dataset[dependent_variable].unique()
    for label in labels:
        temp = dataset[dependent_variable].value_counts()[label] / len(dataset[dependent_variable])
        entropy += -temp * log(temp)
    return entropy

In [72]:
def entropy_on_attribute_split(dataset, attribute):
    """ Finds resulting entropy of dataset if it is split using attribute """
    entropy_after_split = 0
    dependent_variable = dataset.keys()[-1]
    class_labels = dataset[dependent_variable].unique()
    attribute_labels = dataset[attribute].unique()
    
    for attribute_label in attribute_labels:
        entropy = 0
        for class_label in class_labels:
            numer = len(dataset[attribute][dataset[attribute] == attribute_label][dataset[dependent_variable] == class_label])
            denom = len(dataset[attribute][dataset[attribute] == attribute_label])
            temp = numer / (denom + ps)
            entropy += -temp * log(temp + ps)
        temp2 = denom / len(dataset)
        entropy_after_split += -temp2 * entropy
    return abs(entropy_after_split)

In [73]:
def split_criteria(dataset):
    """ Finds the best attribute to split on """
    
    # Define categorical attributes
    attributes = ["Work_accident", "promotion_last_5years", "sales", "salary"]    
    Info_gain = {"Work_accident" : 0, "promotion_last_5years" : 0, "sales" : 0, "salary" : 0}
    for key in dataset.keys()[:-1]:
        if key in attributes:
            Info_gain[key] = entropy_before_split(dataset) - entropy_on_attribute_split(dataset, key)
    return max(Info_gain, key=lambda k: Info_gain[k])

In [75]:
def split_dataset(dataset, feature, label):
    """ This splits the dataset on given feature and all of its values """
    return dataset[dataset[feature] == label].reset_index(drop = True)

In [ ]:
def Decision_tree(train_data):
    """ Builds tree recursively """
    D_tree = {}
    dependent_variable = train_data.keys()[-1]
    root = split_criteria(train_data)
    labels = train_data[root].unique()
    D_tree[node] = {}
    
    for label in labels:
        split_data = split_dataset(train_data, )
    